<a href="https://colab.research.google.com/github/DiegoRomanCortes/AED/blob/pr%2F1/Tarea4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CC3001 Otoño 2020 Tarea 4
# Generación de código

Nombre: Diego Román

Profesores: Sección 1 Patricio Poblete / Sección 2 Nelson Baloian

Plazo: Lunes 8 de junio de 2020 a las 23:59

El objetivo de esta tarea es aprender sobre la manipulación de árboles que representan fórmulas. Hay muchas cosas que uno puede hacer con estos árboles, y en esta tarea nos enfocaremos en particular en algo que los compiladores hacen: la **generación de código** para evaluar fórmulas.

Específicamente, el problema es: dada una fórmula, generar una secuencia de instrucciones elementales que la evalúen. Por ejemplo, dada una fórmula como

```
(a+b)*(c-d)
```

se podría generar código como el siguiente:

```
t1=a+b
t2=c-d
t3=t1*t2
```

Las instrucciones elementales son siempre de la forma ``ti=x op y``, donde ``ti`` es una variable temporal, ``x`` e ``y`` pueden ser variables o números, y ``op`` es un operador. Uno va creando variables ``t1``, ``t2``, ... a medida que las necesita, y la última de ellas es la que almacena el valor final de la fórmula.

Las fórmulas pueden ser arbitrariamente complicadas, como por ejemplo la fórmula

$$
(2-p\,q)\left( \frac{1}{n}+\frac{1}{p+q}\right)
$$

la cual escribimos como

```
(2-p*q)*(1/n+1/(p+q))
```

y cuyo código generado podría ser,

```
t1=p*q
t2=2-t1
t3=1/n
t4=p+q
t5=1/t4
t6=t3+t5
t7=t2*t6
```

El objetivo de esta tarea es que usted implemente una función que, dado un árbol ya construído, lo recorra para generar una secuencia de instrucciones elementales que lo evalúe.

A continuación usted encontrará clases ``Nodoi``, ``Nodoe`` y ``Arbol`` que implementan árboles que representan fórmulas. La clase ``Arbol`` viene con un constructor que recibe un string como parámetro, el cual contiene una fórmula, y este constructor se encarga de transformar la fórmula de string a árbol, de modo que usted no necesita preocuparse de eso. Para simplificar, la fórmula solo podrá contener variables de una sola letra, números de un solo dígito, y sin espacios. Los operadores permitidos son solo suma, resta, multiplicación y división. No hay operador menos unario.

Usted tienen que implementar para la clase ``Arbol`` una función ``codigo`` que al ser invocado genere una secuencia de instrucciones elementales para evaluar la fórmula representada por ese árbol, y la retorne en forma de una lista (de Python).

In [0]:
class Nodoi:
    def __init__(self, izq, info, der):
        self.izq=izq
        self.info=info
        self.der=der
    def postorden(self):
        self.izq.postorden()   
        self.der.postorden()
        print(self.info, end=" ")

class Nodoe:
    def __init__(self, info):
        self.info=info
    def postorden(self):
        print(self.info, end=" ")

class Arbol:
    def __init__(self,formula):
        global k
        global s
        s=formula+";" # agregamos una marca de fin de la entrada
        k=0 # indica próximo caracter por procesar
        # definimos funciones para analizar la fórmula
        def expresion(): # retorna puntero a la raíz de un árbol que representa a la fórmula s
            global k
            global s
            a=factor()
            while s[k]=="+" or s[k]=="-":
                op=s[k]
                k+=1
                b=factor()
                a=Nodoi(a,op,b)
            return a
        def factor():
            global k
            global s
            a=termino()
            while s[k]=="*" or s[k]=="/":
                op=s[k]
                k+=1
                b=termino()
                a=Nodoi(a,op,b)
            return a
        def termino(): # posible constante, variable o formula parentizada
            global k
            global s
            if s[k].isalpha() or s[k].isdigit():
                a=Nodoe(s[k])
                k+=1
                return a
            if s[k]=="(": # fórmula parentizada
                k+=1
                a=expresion()
                if s[k]!=")":
                    print("Error: Falta cierra paréntesis: "+formula[k:])
                    assert False
                k+=1
                return a
            print("Error: Falta variable, número o abre paréntesis: "+formula[k:])
            assert False
              
        a=expresion()
        if s[k]!=";":
            print("Error: Basura al final de la fórmula: "+formula[k:])
            assert False                
        self.raiz=a
            
    def postorden(self):
        print("Postorden:", end=" ")
        self.raiz.postorden()
        print()
    
    def codigo(self):
        global t #contador de llamadas recursivas
        t = 0  #al principio no hay llamadas
        def codigo_rec(p, lista):
          global t
          if p.info.isalpha() or p.info.isnumeric(): #es una hoja/NodoE
            t += 1 #incrementa el contador de llamadas
            nombre = 't'+str(t) #genera la variable
            cod = nombre+'='+p.info #genera la expresion como una asignacion tipo tn = var/num
          elif (p.izq.info.isalpha() or p.izq.info.isnumeric()) \
          and (p.der.info.isalpha() or p.der.info.isnumeric()): #ambos hijos son NodoE
            t += 1
            nombre = 't'+str(t)
            cod = nombre+'='+p.izq.info+p.info+p.der.info #genera la expresion del tipo tn = var/num op var/num
          else: #llamadas recursivas
            a = codigo_rec(p.izq, lista) #tn1
            b = codigo_rec(p.der, lista) #tn2
            t += 1
            nombre = 't'+str(t)
            cod = nombre+'='+a+p.info+b #genera codigo del tipo tn = tn1 op tn2
          lista.append(cod) #en cualquier caso, el codigo generado se guarda en el ultimo lugar de la lista
          return nombre #así las llamadas recursivas incluyen sólo el rótulo y no la expresion de asignacion completa
        #puntapié inicial
        lista = []
        p = self.raiz
        codigo_rec(p, lista)
        return lista #devuelve lo pedido

A continuación probamos esta funcionalidad con varias fórmulas de ejemplo. Para ver si el árbol fue construido correctamente, lo recorremos en postorden para imprimirlo en notación polaca. **Importante**: La notación polaca no tiene nada que ver con lo que ustedes tienen que hacer, es solo una manera de poder imprimir el árbol generado para chequear que esté bien construido.

In [2]:
Arbol("a+b").postorden()

Postorden: a b + 


In [3]:
Arbol("(a+b)*(c-d)").postorden()

Postorden: a b + c d - * 


In [4]:
Arbol("(2-p*q)*(1/n+1/(p+q))").postorden()

Postorden: 2 p q * - 1 n / 1 p q + / + * 


## Resultados

En esta sección probamos la generación de código que usted tiene que escribir. Mientras usted no sustituya la función ``codigo`` provisoria que viene en la definición de la clase, va a aparecer la lista que generamos ahí. Primero vamos a definir una función ``probar`` que genera el código a partir de una fórmula dada e imprime el resultado:

In [0]:
def probar(formula):
    a=Arbol(formula)
    for x in a.codigo():
        print(x)

A continuación probamos con varias fórmulas. Además de las fórmulas dadaas, usted tiene que agregar tres ejemplos adicionales.

In [6]:
probar("a+1")

t1=a+1


In [7]:
probar("(a+b)*(c-d)")

t1=a+b
t2=c-d
t3=t1*t2


In [8]:
probar("(2-p*q)*(1/n+1/(p+q))")

t1=2
t2=p*q
t3=t1-t2
t4=1/n
t5=1
t6=p+q
t7=t5/t6
t8=t4+t7
t9=t3*t8


In [9]:
probar("((((a+b)+c)+d)+e)")

t1=a+b
t2=c
t3=t1+t2
t4=d
t5=t3+t4
t6=e
t7=t5+t6


In [10]:
probar("(a+(b+(c+(d+e))))")

t1=a
t2=b
t3=c
t4=d+e
t5=t3+t4
t6=t2+t5
t7=t1+t6


In [11]:
probar("a")

t1=a


Ejemplos adicionales

In [12]:
probar("y*3-1*z+w")

t1=y*3
t2=1*z
t3=t1-t2
t4=w
t5=t3+t4


In [13]:
probar('(1-2)*a-(b/3)')

t1=1-2
t2=a
t3=t1*t2
t4=b/3
t5=t3-t4


In [14]:
probar('a+b/(c+1*d)-2/e')

t1=a
t2=b
t3=c
t4=1*d
t5=t3+t4
t6=t2/t5
t7=t1+t6
t8=2/e
t9=t7-t8


In [15]:
probar('a/b*c')

t1=a/b
t2=c
t3=t1*t2


## Discusión

Suponga que hay escasez de variables temporales, así que se le ha pedido que trata de reciclarlas lo más posible, y solo crear nuevas variables ``ti`` cuando sea estrictamente necesario. Por ejemplo, para la fórmula

```
((a+b)*c)+((d-e)/f)
```

si generamos código como lo hemos hecho en los ejemplos anteriores (código de la izquierda), se usan 5 variables temporales, pero el código de la derecha muestra que se puede evaluar usando solo 2:

```
t1=a+b               t1=a+b
t2=t1+c              t1=t1*c
t3=d-e               t2=d-e
t4=t3/f              t2=t2/f
t5=t2+t4             t1=t1+t2
```

Discuta cómo se podría modificar su algoritmo para que (1) use menos variables temporales, y luego (2) para que use el mínimo posible de variables temporales.

No es necesario que implemente nada, solo que discuta cómo se podría hacer.

* (1) Por cómo está programada la función `codigo`, ésta genera una nueva variable temporal cada vez que se encuentra una hoja solitaria. Esto podría solventarse separando en tres casos: El primero cuando el hijo izquierdo sea `Nodoe`, el segundo cuando el derecho lo sea y finalmente el caso en que ambos lo sean. Si junto con eso además se reutilizan las variables temporales que no se vayan a seguir usando, es posible vislumbrar un posible respuesta a (2).
* Notando que la generación de nuevas variables temporales es necesaria solamente cuando hay operadores distintos con más de un operando permite reutilizar las variables. Por ejemplo, `'(a-b)/(c+d)'` debería generar: `t1 = a-b`,`t2=c+d`, `t1 = t1 / t2`. Un árbol lleno con $n$ niveles y con operadores distintos requiere un mínimo de $n$ variables temporales en el peor caso (operadores alternando prioridad). 
* Por lo tanto, para minimizar las variables temporales (2) se debe considerar la creación de un máximo de $n$ variables para un árbol de $n$ niveles (número que disminuye si las operaciones se pueden asociar o si hay hojas solitarias), además de incluir nuevos casos: Que la resta de una suma sea una doble resta, que la división de un producto sea una doble división, que al llegar a la raiz del árbol las variables generadas distintas a `t1` se puedan reutilizar (reiniciando el contador), que el último paso sea `t1 = t1 op t2`, entre otros.



